# Cas Kaggle ApC
## Participants
Martí Armengod Villar 

Joan Marc Samó Rojas

## Objectius

https://www.kaggle.com/datasets/datasnaek/chess


En la nostra Base de dades el que volem és ser capaços de predir quin jugador guanyara a partir de la seva llencçada inicial.



Ara el que farem es importar les nostres dades i separar-les entre Train i Test per tal de evitar filtració de dades 

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split

dades = pd.read_csv('games.csv')

train_set, test_set = train_test_split(dades,random_state=42,test_size=0.2)
X = train_set['data']
y = train_set['target']




In [15]:
tipus_atributs = train_set.dtypes
print(tipus_atributs)


id                 object
rated                bool
created_at        float64
last_move_at      float64
turns               int64
victory_status     object
winner             object
increment_code     object
white_id           object
white_rating        int64
black_id           object
black_rating        int64
moves              object
opening_eco        object
opening_name       object
opening_ply         int64
dtype: object


Hem volgut assegurar-nos de si les nostres dades tenien o no Nans i tal i com haviem observat en la web del Kaggle no en tenim. Ho podem observar en la següent funcio de comprovacio de Nans, Cap columna te cap Nan.

In [12]:
nans = train_set.isna().sum()

print(nans)

id                0
rated             0
created_at        0
last_move_at      0
turns             0
victory_status    0
winner            0
increment_code    0
white_id          0
white_rating      0
black_id          0
black_rating      0
moves             0
opening_eco       0
opening_name      0
opening_ply       0
dtype: int64


Ara el farem es veure quines columnes són les que necesitem per poder predir el que volem saber que és qui guanyara segons els primers moviments.

Hem de tenir en compte que tenim columnes com per exemple la id de la partida que no ens aporta cap dada rellevant ja que es unica per cada partida i no podriem obtenir informació d'ella.

També ens hem donat compte que tenim files identiques repetides multiples vegades aixi que avanç de res el que farem es eliminar les repeticons asegurant-nos de eliminar nomes les que son 100% iguals.

In [28]:
# Elimina files duplicades completament iguals
dades_sense_duplicats = dades.drop_duplicates()

print(f"Abans de treure duplicats: {dades.shape[0]} files")
print(f"Després de treure duplicats: {dades_sense_duplicats.shape[0]} files")
print(dades_sense_duplicats)


Abans de treure duplicats: 20058 files
Després de treure duplicats: 19629 files
             id  rated    created_at  last_move_at  turns victory_status  \
0      TZJHLljE  False  1.504210e+12  1.504210e+12     13      outoftime   
1      l1NXvwaE   True  1.504130e+12  1.504130e+12     16         resign   
2      mIICvQHh   True  1.504130e+12  1.504130e+12     61           mate   
3      kWKvrqYL   True  1.504110e+12  1.504110e+12     61           mate   
4      9tXo1AUZ   True  1.504030e+12  1.504030e+12     95           mate   
...         ...    ...           ...           ...    ...            ...   
20053  EfqH7VVH   True  1.499791e+12  1.499791e+12     24         resign   
20054  WSJDhbPl   True  1.499698e+12  1.499699e+12     82           mate   
20055  yrAas0Kj   True  1.499698e+12  1.499698e+12     35           mate   
20056  b0v4tRyF   True  1.499696e+12  1.499697e+12    109         resign   
20057  N8G2JHGG   True  1.499643e+12  1.499644e+12     78           mate   

      w

In [ ]:
dades_sense_columna = dades_sense_duplicats.drop(columns=["created_at","last_move_at"])
dades_sense_duplicats_sense_columna = dades.drop_duplicates()

#print(dades_sense_columna)
print(f"Abans de treure duplicats: {dades_sense_duplicats_sense_columna.shape[0]} files")



# Seleccionar totes les files amb id XXXX
id_seleccionat = "XRuQPSzH"
files_seleccionades = dades_sense_columna[dades_sense_columna["id"] == id_seleccionat]

print(files_seleccionades)

Abans de treure duplicats: 19629 files
             id  rated  turns victory_status winner increment_code  \
120    XRuQPSzH   True     50         resign  black           10+0   
12412  XRuQPSzH   True     50         resign  black           10+0   

          white_id  white_rating black_id  black_rating  \
120    chess-brahs          2036     cdvh          1706   
12412  chess-brahs          2036     cdvh          1706   

                                                   moves opening_eco  \
120    Nf3 d5 g3 c5 Bg2 Nc6 d3 Nf6 O-O e5 Nc3 d4 Ne4 ...         A08   
12412  Nf3 d5 g3 c5 Bg2 Nc6 d3 Nf6 O-O e5 Nc3 d4 Ne4 ...         A08   

                                 opening_name  opening_ply  
120    King's Indian Attack: French Variation            6  
12412  King's Indian Attack: French Variation            6  
